In [2]:
import argparse
import os
import copy
os.environ['http_proxy'] = "http://10.176.52.116:7890"
os.environ['https_proxy'] = "http://10.176.52.116:7890"
os.environ['all_proxy'] = "socks5://10.176.52.116:7891"
import sys
sys.path.append(('../'))
sys.path.append(('../../'))
# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import json
from typing import Dict
import torch
from torch.nn import CrossEntropyLoss
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from task_vector import TaskVector

from watermarks.kgw.watermark_processor import WatermarkDetector
from watermarks.aar.aar_watermark import AarWatermarkDetector
from watermarks.watermark_types import WatermarkType
import logging
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

### Load in MLP

In [3]:
class ResidualBlock(nn.Module):
    def __init__(self, dim):
        super(ResidualBlock, self).__init__()
        self.fc = nn.Linear(dim, dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc(x)
        out = self.relu(out)
        out = out + x 
        return out

class TransformModel(nn.Module):
    def __init__(self, num_layers=4, input_dim=1024, hidden_dim=500, output_dim=300):
        super(TransformModel, self).__init__()
        
        self.layers = nn.ModuleList()
        
        self.layers.append(nn.Linear(input_dim, hidden_dim))
        
        for _ in range(num_layers - 2):
            self.layers.append(ResidualBlock(hidden_dim))

        self.layers.append(nn.Linear(hidden_dim, output_dim))

    def forward(self, x):
        for i in range(len(self.layers)):
            x = self.layers[i](x)
        return x
    
save_path = '/remote-home/miintern1/watermark-learnability/data/model_weights_2/model.layers.31_kgw-k0-gamma0.25-delta1_hidden_dim_8192_num_layers_1_best_model.pt'
hyperparameter_dict = {
    'num_layers': 4,
    'input_dim': 4096,
    'hidden_dim': 500,
    'output_dim': 300
}
loaded_dict = torch.load(save_path, map_location='cpu')
model_state_dict = loaded_dict["model_state_dict"]
loaded_hyperparameters = loaded_dict["hyperparameters"]

loaded_num_layers = loaded_hyperparameters["num_layers"]
loaded_hidden_dim = loaded_hyperparameters["hidden_dim"]
loaded_learning_rate = loaded_hyperparameters["learning_rate"]
loaded_num_epochs = loaded_hyperparameters["num_epochs"]

loaded_model = TransformModel(
    num_layers=loaded_num_layers,
    input_dim=4096,  # Change this as needed
    hidden_dim=loaded_hidden_dim,
    output_dim=4096  # Change this as needed
)

# Load the model state dictionary into the initialized model
loaded_model.load_state_dict(model_state_dict)

<All keys matched successfully>

### Generation imitation

In [58]:
loaded_model.half()

TransformModel(
  (layers): ModuleList(
    (0): Linear(in_features=4096, out_features=8192, bias=True)
    (1): Linear(in_features=8192, out_features=4096, bias=True)
  )
)

In [59]:
# Function to get layer output and substitute with transformed activation
def substitute_layer_hook(layer_name, mlp_model=None):
    def hook_fn(module, input, output):
        try:
            # Store the original output
            if layer_name not in vanilla_outputs:
                vanilla_outputs[layer_name] = []
            activation = output[0][0]
            # Debug: Print activation dtype
            print(f"Inside hook: {layer_name}, {activation.dtype=}")
            vanilla_outputs[layer_name].append(output)

            # If mlp_model is provided, use it to transform the output
            if mlp_model is not None:
                print(f"Original: {activation.dtype=}")
                print(f"Transforming activation with MLP model")
                # with torch.no_grad():
                # Ensure the output is on the same device as the mlp_model
                activation = output[0]
                transformed_output = mlp_model(activation.cpu()).to(torch.float16).to(output[0].device)
                print(f"Transformed: {transformed_output.dtype=}")
                modified_output = (transformed_output, *output[1:])
                if layer_name not in modified_outputs:
                    modified_outputs[layer_name] = []
                modified_outputs[layer_name].append(modified_output[0].cpu())
                return modified_output
            else:
                print(f"Original: {activation.dtype=} without transformation")
                return output
        except Exception as e:
            print(f"Error in hook {layer_name}: {e}")
            return output
    return hook_fn


# Example of usage
vanilla_outputs = {}
modified_outputs = {}
watermark_outputs = {}
hook_layer = [31]  # Specify the layers to hook

# Load your vanilla model
hooked_model_name = "meta-llama/Llama-2-7b-hf"
hooked_model = AutoModelForCausalLM.from_pretrained(hooked_model_name)
tokenizer = AutoTokenizer.from_pretrained(hooked_model_name)
hooked_model.half()
 
# Dictionary to store hook handles
hook_handles = {}

# Register hooks to the specified layers
for i in hook_layer:
    layer_name = f"model.layers.{i}"
    layer = dict([*hooked_model.named_modules()])[layer_name]
    hook_handle = layer.register_forward_hook(substitute_layer_hook(layer_name, loaded_model))
    hook_handles[layer_name] = hook_handle


# Function to remove hooks
def remove_hooks(hook_handles):
    for handle in hook_handles.values():
        handle.remove()

# At some point in your code, when you want to remove the hooks


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [60]:
# Input text
# remove_hooks(hook_handles)
text = "Please introduce yourself to me:"
inputs = tokenizer(text, return_tensors='pt', truncation=True)
input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']


# Run the model to activate hooks
with torch.no_grad():
    hooked_logit = hooked_model(input_ids, attention_mask=attention_mask, return_dict=True).logits

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Inside hook: model.layers.31, activation.dtype=torch.float16
Original: activation.dtype=torch.float16
Transforming activation with MLP model
Transformed: transformed_output.dtype=torch.float16


In [26]:
len(vanilla_outputs['model.layers.31'])

3

In [ ]:
def get_layer_output_hook(layer_name):
    def hook_fn(module, input, output):
        # print(f"{output[0].shape=}")
        # print(f"{output[1]=}")
        if layer_name not in watermark_outputs:
            watermark_outputs[layer_name] = []
        watermark_outputs[layer_name].append(output[0][:, -1, :].cpu())
    return hook_fn

watermark_model_name = "cygu/llama-2-7b-logit-watermark-distill-kgw-k0-gamma0.25-delta1"
watermarked_model = AutoModelForCausalLM.from_pretrained(watermark_model_name)

for i in hook_layer:
    layer_name = f"model.layers.{i}"
    layer = dict([*watermarked_model.named_modules()])[layer_name]
    layer.register_forward_hook(get_layer_output_hook(layer_name))


vanilla_model_name = "meta-llama/Llama-2-7b-hf"
vanilla_model = AutoModelForCausalLM.from_pretrained(vanilla_model_name)
tokenizer = AutoTokenizer.from_pretrained(vanilla_model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [47]:
# Input text
text = "Please introduce yourself to me:"
inputs = tokenizer(text, return_tensors='pt', truncation=True)
input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']


# Run the model to activate hooks
with torch.no_grad():
    hooked_logit = hooked_model(input_ids, attention_mask=attention_mask, return_dict=True).logits

# Run the model to activate hooks
with torch.no_grad():
    output_logit = vanilla_model(input_ids, attention_mask=attention_mask, return_dict=True).logits

# Run the model to activate hooks
with torch.no_grad():
    watermark_logit = watermarked_model(input_ids, attention_mask=attention_mask, return_dict=True).logits

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [72]:
substututed_output = hooked_model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    do_sample=True,  # or False, depending on your requirement
    min_length=50,
    max_length=100,
    temperature=1.0,
    top_p=0.95,
    top_k=50,
    pad_token_id=tokenizer.eos_token_id,
)

substututed_text = tokenizer.decode(substututed_output[0], skip_special_tokens=True)
print(substututed_text)
print('-'*100)

generated_output = vanilla_model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    do_sample=True,  # or False, depending on your requirement
    min_length=50,
    max_length=100,
    temperature=1.0,
    top_p=0.95,
    top_k=50,
    pad_token_id=tokenizer.eos_token_id,
)

generated_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)
print(generated_text)
print('-'*100)

watermarked_output = watermarked_model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    do_sample=True,  # or False, depending on your requirement
    min_length=50,
    max_length=100,
    temperature=1.0,
    top_p=0.95,
    top_k=50,
    pad_token_id=tokenizer.eos_token_id,
)
watermarked_text = tokenizer.decode(watermarked_output[0], skip_special_tokens=True)
print(watermarked_text)
print('-'*100)

Please introduce yourself to me:
[name], a person who is not a student, a teacher, an employee or a parent
A good-looking woman
Please explain how you would answer the questions differently if you were an employer:
A better-looking woman with bigger breasts
The answers are the same. If you think an answer is too personal for a parent to ask you, the answer is probably too personal for a teacher or student to ask you. And the same
----------------------------------------------------------------------------------------------------
Please introduce yourself to me:
Hey, I’m Siobhan! I’m a 17-year-old photographer who recently graduated high school, and I’m currently studying photography at the University of Lincoln.
Who or what inspires you and why?
There’s a lot of things that inspire me. A lot of photographers inspire me, artists like Diane Arbus and Nan Goldin really inspired me as a younger
------------------------------------------------------------------------------------------------

In [73]:
detector = WatermarkDetector(
                            device='cpu',
                            tokenizer=tokenizer,
                            vocab=tokenizer.get_vocab().values(),
                            gamma=0.25,
                            seeding_scheme='simple_0',
                            normalizers=[],
                        )
waterk_score = detector.detect(watermarked_text)
substitution_score = detector.detect(substututed_text)
generated_score = detector.detect(generated_text)

print(f"Watermarked score: {waterk_score['p_value']}")
print(f"Substituted score: {substitution_score['p_value']}")
print(f"Generated score: {generated_score['p_value']}")


Watermarked score: 0.011818979440471876
Substituted score: 0.08319032287015378
Generated score: 0.49219720463171884


In [ ]:
watermarked_model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    cache_dir="/path/to/cache/dir",  # Optional: specify a cache directory
    force_download=False,            # Optional: force a fresh download
    local_files_only=True            # Ensure it only looks locally
)


In [86]:
max_length = 250
min_length = 250
num_samples = 512
batch_size = 1
save_path = ''


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed(42)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def filter_length(example):
    return len(tokenizer(example['text'], truncation=True, max_length=max_length)["input_ids"]) >= min_length

def encode(examples):
    trunc_tokens = tokenizer(
        examples['text'],
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors="pt"
    ).to(device)
    # Examples are truncated to max_length, which comprises the possible generation prompt and the text to be generated
    examples["text"] = tokenizer.batch_decode(trunc_tokens["input_ids"], skip_special_tokens=True)
    prompt = tokenizer(
        examples["text"], truncation=True, padding=True, max_length=50, return_tensors="pt",
    ).to(device)
    examples["prompt_text"] = tokenizer.batch_decode(prompt["input_ids"], skip_special_tokens=True)
    examples["input_ids"] = prompt["input_ids"]
    examples["attention_mask"] = prompt["attention_mask"]
    examples["text_completion"] = tokenizer.batch_decode(
        trunc_tokens["input_ids"][:, 50:], skip_special_tokens=True
    )
    return examples

dataset = dataset.filter(filter_length)
# Set how many samples will be skipped
dataset = dataset.map(encode, batched=True)
dataloader = torch.utils.data.DataLoader(dataset, batch_size)

prompts = []
human_text = []
prompt_text = []
full_human_text = []
for batch in dataloader:
    if len(human_text) >= num_samples:
        break
    if (type(batch["input_ids"]) == list):
        batch["input_ids"] = torch.stack(batch["input_ids"], dim=1).to(device)
    if (type(batch["attention_mask"]) == list):
        batch["attention_mask"] = torch.stack(batch["attention_mask"], dim=1).to(device)
    prompts.append(batch)
    human_text.extend(batch["text_completion"])
    prompt_text.extend(batch["prompt_text"])
    full_human_text.extend(batch["text"])
human_text = human_text[:num_samples]
prompt_text = prompt_text[:num_samples]
full_human_text = full_human_text[:num_samples]
raw_input = {
    "prompts": prompts,
    "human_text": human_text,
    "prompt_text": prompt_text,
    "full_human_text": full_human_text,
}

In [88]:

def compute_p_value(samples, detector, type='kgw'):
    score_list = []
    for s in tqdm(samples):
        score = detector.detect(s)
        score_list.append(score['p_value']) if type=='kgw' else score_list.append(score)
    return score_list


def compute_seq_rep_n(samples, tokenizer, n=3):
    """compute seq-rep-n metric"""
    n_gram_reps = []
    
    for s in samples:
        n_grams = []
        tokens = tokenizer(s, add_special_tokens=False).input_ids
        for i in range(len(tokens)):
            if i <= len(tokens) - n:
                n_grams.append(tuple(tokens[i:i + n]))
                    
        rep = 1 - len(set(n_grams)) / len(n_grams)
        n_gram_reps.append(rep)
            
    median_rep = np.median(n_gram_reps)
    mean_rep = np.mean(n_gram_reps)
    return {
        f"median_seq_rep_{n}": median_rep,
        f"mean_seq_rep_{n}": mean_rep,
        f"list_seq_rep_{n}": n_gram_reps,
    }


def compute_total_rep_n(samples, tokenizer, n=3):
    """compute total-rep-n metric"""
    n_grams = []
    
    for s in samples:
        tokens = tokenizer(s, add_special_tokens=False).input_ids
        for i in range(len(tokens)):
            if i <= len(tokens) - n:
                n_grams.append(tuple(tokens[i:i + n]))

    total_rep = 1 - len(set(n_grams)) / len(n_grams)        

    return {f"total_rep_{n}": total_rep}


def compute_repetition(samples_dict, tokenizer):
    """Compute repetition metrics."""
    samples = samples_dict['truncate_prompt_output']
    samples_dict.update(compute_seq_rep_n(samples, tokenizer, n=3))
    samples_dict.update(compute_total_rep_n(samples, tokenizer, n=3))
    # print(f"Model name: {model_name}\nMedian seq rep 3: {samples['median_seq_rep_3']}\nTotal rep 3: {samples['total_rep_3']}")
    return f"Median seq rep 3: {samples_dict['median_seq_rep_3']}\nTotal rep 3: {samples_dict['total_rep_3']}"

def compute_ppl(samples_dict, prompts,  tokenizer, model, batch_size, fp16=True):
    """Compute perplexities under `ppl_model_name`."""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if model.device.type != device:
        original_device = model.device
        model.to(device)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    ppls = []
    loss_fct = CrossEntropyLoss(reduction="none")

    samples = samples_dict["full_output"]

    for i in tqdm(range(0, len(samples), batch_size)):
        s = samples[i:i + batch_size]
        encodings = tokenizer(
            s,
            add_special_tokens=True,
            padding=True,
            return_tensors="pt",
            return_attention_mask=True,
        ).to(device)

        encoded_batch = encodings["input_ids"]
        attn_mask = encodings["attention_mask"]

        labels = encoded_batch

        with torch.no_grad():
            out_logits = model(encoded_batch, attention_mask=attn_mask).logits

        prompt_text = prompts[i:i + batch_size]
        # print(prompt_text)
        # print(type(prompt_text))
        # print(len(prompt_text))
        
        prompt_encodings = tokenizer(
            prompt_text,
            add_special_tokens=True,
            padding=True,
            return_tensors="pt",
            return_attention_mask=True,
        ).to(device)
        prompt_attn_mask = prompt_encodings["attention_mask"]

        # match shape of prompt_attn_mask and attn_mask by padding with 0
        padding = torch.zeros(
            (attn_mask.shape[0], attn_mask.shape[1] - prompt_attn_mask.shape[1]),
        ).to(device)
        padded_prompt_attn_mask = torch.cat([prompt_attn_mask, padding], dim=1)
        prompt_mask = (padded_prompt_attn_mask == 1)
        
        # don't score prompt tokens
        attn_mask[prompt_mask] = 0

        shift_logits = out_logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        shift_attention_mask_batch = attn_mask[..., 1:].contiguous()

        perplexity_batch = torch.exp(
            (loss_fct(shift_logits.transpose(1, 2), shift_labels) * shift_attention_mask_batch).sum(1)
            / shift_attention_mask_batch.sum(1)
        )

        ppls += perplexity_batch.tolist()

    mean_perplexity = np.mean(ppls)
    median_perplexity = np.median(ppls)
    samples_dict["mean_perplexity"] = mean_perplexity
    samples_dict["median_perplexity"] = median_perplexity
    samples_dict["perplexities"] = ppls
    # if original_device!=device:
    #     model.to(original_device)
    return f"mean perplexity: {mean_perplexity}, median perplexity: {median_perplexity}"


def move_to_device(batch, device):
    """Move batch to the specified device."""
    new_batch = {}
    for key, value in batch.items():
        if isinstance(value, torch.Tensor):
            new_batch[key] = value.to(device)
        elif isinstance(value, list):
            # Assuming lists are lists of tensors, move each tensor to the device
            new_batch[key] = [v.to(device) if isinstance(v, torch.Tensor) else v for v in value]
        else:
            new_batch[key] = value
    return new_batch


In [89]:

def generate_and_evaluate(prompts, num_samples, vanilla_model, tokenizer, generation_result_dict, watermark_config):
    output_results = []
    full_output_results = []

    for batch in tqdm(prompts):
        if len(output_results) >= num_samples:
            break
        with torch.no_grad():
            batch = move_to_device(batch, "cpu")
            vanilla_output = vanilla_model.generate(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                do_sample=True,
                min_length=50,
                max_length=100,
                temperature=1.0,
                top_p=0.95,
                top_k=50,
                pad_token_id=tokenizer.eos_token_id,
            )
        
            n_input_tokens = batch["input_ids"].shape[1]
            truncated_prompt_output = vanilla_output[:, n_input_tokens:]

            output_results.extend(tokenizer.batch_decode(truncated_prompt_output, skip_special_tokens=True))
            full_output_results.extend(tokenizer.batch_decode(vanilla_output, skip_special_tokens=True))

        generation_result_dict['full_output'] = full_output_results[:num_samples]
        generation_result_dict["truncate_prompt_output"] = output_results[:num_samples]
     

        output_results = output_results[:num_samples]
        if watermark_config["type"] == "kgw":
            detector = WatermarkDetector(
                device=watermark_config.get("kgw_device", 'cpu'),
                tokenizer=tokenizer,
                vocab=tokenizer.get_vocab().values(),
                gamma=watermark_config["gamma"],
                seeding_scheme=watermark_config["seeding_scheme"],
                normalizers=[],
            )
        elif watermark_config["type"] == "aar":
            detector = AarWatermarkDetector(tokenizer=tokenizer, k=watermark_config['k'], seed=watermark_config['seed'], eps=1e-20)
        
        output_scores = compute_p_value(output_results, detector, type=watermark_config["type"])
       
        generation_result_dict['watermark_scores'] = output_scores
        rep_output = compute_repetition(generation_result_dict, tokenizer)
        print(f"{rep_output}")
        ppl_output = compute_ppl(generation_result_dict, prompt_text, tokenizer, vanilla_model, batch_size)
        print(f"{ppl_output}")
    return generation_result_dict

generation_dict = {}
result = generate_and_evaluate(prompts, 2, vanilla_model, tokenizer, generation_dict, {'type': 'kgw', 'gamma': 0.25, 'seeding_scheme': 'simple_0'})

100%|██████████| 1/1 [00:00<00:00, 29.23it/s]


Median seq rep 3: 0.0
Total rep 3: 0.0


  0%|          | 1/512 [00:21<3:03:09, 21.51s/it]

mean perplexity: 3.885118246078491, median perplexity: 3.885118246078491


100%|██████████| 2/2 [00:00<00:00, 30.63it/s]


Median seq rep 3: 0.0
Total rep 3: 0.0


  0%|          | 2/512 [00:44<3:08:06, 22.13s/it]

mean perplexity: 4.476223111152649, median perplexity: 4.476223111152649


In [93]:
generation_dict

{'full_output': ["Cluster comprises IBM's Opteron-based eServer 325 server and systems management software and storage devices that can run Linux and Windows operating systems.\nIBM on Tuesday announced a prepackaged and pre-priced system of servers, software and storage devices that aims to meet the business computing needs of small and midsize companies, particularly those with Linux servers.\nThe offering, called Cluster 100, includes IBM's Op",
  'Belying expectations, Prasar Bharti has earned only Rs 58.19 crore (Rs 581.9 million) as revenue during the Commonwealth Games last month.\nThe gross revenue, which excludes revenue from sale of television rights and advertising slots, is far lower than its actual income during the Beijing Olympics (Rs 213.21 crore) and the previous Games in'],
 'truncate_prompt_output': ["-priced system of servers, software and storage devices that aims to meet the business computing needs of small and midsize companies, particularly those with Linux ser